In [186]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


You should consider upgrading via the 'D:\projects\vsCode\pythonForHtests\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [187]:
!pip3 install -q "monai-weekly[nibabel, skimage, pillow, tensorboard, gdown, ignite, torchvision, itk, tqdm, lmdb, psutil,  openslide, pandas, einops, transformers, mlflow, matplotlib, tensorboardX, tifffile, imagecodecs]"

You should consider upgrading via the 'D:\projects\vsCode\pythonForHtests\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [188]:
!pip3 install h5py
!python -c "import matplotlib" || pip install -q matplotlib

You should consider upgrading via the 'D:\projects\vsCode\pythonForHtests\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [189]:
import numpy as np

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,

)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

import matplotlib.pyplot as plt


from monai.transforms import (
    LoadImage, LoadImaged, EnsureChannelFirstd,
    Resized,  Compose
)
from monai.config import print_config
import re


In [190]:
from monai.config import print_config
print_config()

MONAI version: 0.9.dev2210
Numpy version: 1.21.5
Pytorch version: 1.10.2+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 1a660e6a7a50e985af5ff76b559baab44175438c
MONAI __file__: d:\projects\vsCode\pythonForHtests\venv\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.8.0
gdown version: 4.4.0
TorchVision version: 0.11.3+cu113
tqdm version: 4.63.0
lmdb version: 1.3.0
psutil version: 5.9.0
pandas version: 1.1.5
einops version: 0.4.1
transformers version: 4.17.0
mlflow version: 1.24.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [191]:
from monai.transforms import *

set_determinism(seed=0)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)


In [192]:
data_dir = "D:\\dataSets\\CTORGmini\\"

train_images = sorted(
    glob.glob(os.path.join(data_dir, "volumes 0-49", "*.nii.gz")))

train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labels", "*.nii.gz")))

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
train_files, val_files = data_dicts[:-9], data_dicts[-9:]


[{'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-0.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-0.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-1.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-1.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-2.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-2.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-3.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-3.nii.gz'}]

In [193]:
def getNumberFromFilePath(dat):
    string = str(dat['image_meta_dict']['filename_or_obj'])
    numb = re.search('.nii', string).span()[0] ## this does what you wanted
    describingNumber = string[(numb-2):(numb)].replace("-","")
    return int(describingNumber)

In [194]:
import h5py
f = h5py.File("D:\\dataSets\\forMainHDF5\\smallLiverDataSet.hdf5", "w")

In [195]:
# https://github.com/Project-MONAI/MONAI/discussions/3906
# below example how to filter out the files without spacing and orientation in metadata 
# >>> from monai.transforms import LoadImaged
# >>> data_dict = LoadImaged(keys="img")({"img": "./volume-20.nii.gz"})
# >>> (data_dict["img_meta_dict"]["qform_code"], data_dict["img_meta_dict"]["sform_code"])
# (array(0, dtype=int16), array(0, dtype=int16))

# >>> data_dict = LoadImaged(keys="img")({"img": "./volume-21.nii.gz"})
# >>> (data_dict["img_meta_dict"]["qform_code"], data_dict["img_meta_dict"]["sform_code"])
# (array(1, dtype=int16), array(1, dtype=int16))


In [196]:
check_ds = Dataset(data=data_dicts, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)

for dat in check_loader:
    describingNumber = getNumberFromFilePath(dat)
    grp = f.create_group(str(describingNumber))

    labelBoolTensor =torch.where( dat['label']==1, 1, 0).bool()
    summ= torch.sum(labelBoolTensor)
    #if summ>0:
    labelBoolDataset = grp.create_dataset('liver', data=  (labelBoolTensor[0,0,:,:,:]).detach().numpy())
    imageDataset = grp.create_dataset('image', data=  (dat['image'][0,0,:,:,:]).detach().numpy())
    labelBoolDataset.attrs['sum']=  summ
    
    print( labelBoolTensor[0,0,:,:,:].size() )

    imageDataset.attrs['dataType']=  "CT"
    labelBoolDataset.attrs['dataType']=  "boolLabel"

    #seting spacing - needs to be the same for all of the arrays in a grop - so the same for all images in patient
    grp.attrs['spacing']=(1,1,1)

    #get minimum and maximum values
    imageDataset.attrs['min']=torch.min(dat['image'])
    imageDataset.attrs['max']=torch.max(dat['image'])

    labelBoolDataset.attrs['min']=0
    labelBoolDataset.attrs['max']=1

  

    print(describingNumber)
    pass

torch.Size([348, 250, 371])
0
torch.Size([346, 299, 611])
1
torch.Size([397, 358, 517])
2
torch.Size([355, 349, 534])
3


In [197]:
f.close()